In [ ]:
from sympy.physics.quantum.spin import Rotation
from numpy import *
from sympy import pi, symbols
import sympy
from sympy import I, sin, cos, exp
import transformations as tra

In [ ]:
def f_App_A2(l1,l2,m,ri,rj):
    f = (-1)**(m+l2) \
    *math.factorial(l1+l2) \
    *(math.factorial(l1+abs(m)) \
     *math.factorial(l1-abs(m)) \
     *math.factorial(l2+abs(m)) \
     *math.factorial(l2-abs(m)) )**(-.5) \
    *ri**l1 *rj**l2
    return f

def my_euler_matrix(omega):
    alpha = omega[0]
    beta = omega[1]
    gamma = omega[2]
    Re = tra.euler_matrix(alpha,beta,gamma, 'rxyz')
    return matrix(Re)

def my_euler_from_matrix(Re):
    al, be, ga = tra.euler_from_matrix(Re, 'rxyz')
    omega = [al,be,ga]
    return omega

# omega = [0, pi/2, 0]
# Re = my_euler_matrix(omega)
# newomega = my_euler_from_matrix(Re)
# print newomega

In [ ]:
def expand_in_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,R):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = Rotation.D(l1, m, 0,omega1i_bar[0], omega1i_bar[1],0).doit()
                D2j = Rotation.D(l2, m, 0,omega2j_bar[0], omega2j_bar[1],0).doit()
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    return rij


In [ ]:
def expand_in_omega_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = 0
                for mp in range(-l1,l1+1):
                    temp1 = Rotation.D(l1, m, mp, omega1[0], omega1[1], omega1[2]).doit()
                    temp2 = Rotation.D(l1, mp, 0, omega1i_bar[0], omega1i_bar[1], omega1i_bar[2]).doit()
                    D1i += temp1*temp2
                D2j = 0
                for mp in range(-l2,l2+1):
                    temp1 = Rotation.D(l2, m, mp, omega2[0], omega2[1], omega2[2]).doit()
                    temp2 = Rotation.D(l2, mp, 0, omega2j_bar[0], omega2j_bar[1], omega2j_bar[2]).doit()
                    D2j += temp1*temp2
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    return rij


In [ ]:
def expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R,psi):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = 0
                for mp in range(-l1,l1+1):
                    temp1 = 0
                    for mpp in range(-l1,l1+1):
                        tempp1 = Rotation.D(l1, m, mpp,  0,psi,0).doit()
                        tempp2 = Rotation.D(l1, mpp, mp, omega1[0], omega1[1], omega1[2]).doit()
                        temp1 += tempp1*tempp2
                    temp2 = Rotation.D(l1, mp, 0, omega1i_bar[0], omega1i_bar[1], omega1i_bar[2]).doit()
                    D1i += temp1*temp2
                D2j = 0
                for mp in range(-l2,l2+1):
                    temp1 = 0
                    for mpp in range(-l2,l2+1):
                        tempp1 = Rotation.D(l2, m, mpp,  0,psi,0).doit()
                        tempp2 = Rotation.D(l2, mpp, mp, omega2[0], omega2[1], omega2[2]).doit()
                        temp1 += tempp1*tempp2
                    temp2 = Rotation.D(l2, mp, 0, omega2j_bar[0], omega2j_bar[1], omega2j_bar[2]).doit()
                    D2j += temp1*temp2
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    return rij, brackets




In [ ]:
# Specify the order of the expansion
l1max = 1
l2max = 1

In [ ]:
# Now specify the molecule-fixed configuration
small = 1e-3

beta1i = .1;     alpha1i = .1;  ri = 1
omega1i_bar = [alpha1i,beta1i,0]; 
beta2j = .1;     alpha2j = .1;  rj = 1
omega2j_bar = [alpha2j,beta2j,0]; 

# Give the components of omegas as numerical values
alpha1_num = 0
beta1_num = .4*pi
gamma1_num = 0
alpha2_num = 0
beta2_num = .6*pi
gamma2_num = 0

# And as symbolic values
alpha1_sym, beta1_sym, gamma1_sym = symbols('alpha1_sym beta1_sym gamma1_sym', real=True)
alpha2_sym, beta2_sym, gamma2_sym = symbols('alpha2_sym beta2_sym gamma2_sym', real=True)

# Give the r-frame coordinate as numerical values
R_num = 10
psi_num = pi/4
psi_num = 0  # When zero, the expansion with psi should equal the expansion without

# And as symbolic values
R_sym, psi_sym = symbols('R_sym psi_sym', real=True)

In [ ]:
# This is the original formulation

# This is the position of atoms in the molecule-fixed frames (molecule 1 and 2)
R = R_num

# Expand
rij = expand_in_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,R)
#rij = rij.expand(complex=True)
rij = eval(str(rij))
print rij

In [ ]:
# Here, applying a rotation to each frame first

# This relates the two frames
R = R_num
#print 'omega1_bar', omega1i_bar

# Specify rotations of each frame
omega1 = [alpha1_num,beta1_num,gamma1_num]; #print 'omega1', omega1
omega2 = [alpha2_num,beta2_num,gamma2_num]

# Apply it to get a new rotation matrix
R_omega1i_bar = my_euler_matrix(omega1i_bar); #print 'R_omega1i_bar\n', R_omega1i_bar
R_omega2j_bar = my_euler_matrix(omega2j_bar)
R_omega1 = my_euler_matrix(omega1); #print 'R_omega1\n', R_omega1
R_omega2 = my_euler_matrix(omega2)
R_omega1_omega1i_bar = R_omega1*R_omega1i_bar; #print 'R_omega1_omega1i_bar\n', R_omega1_omega1i_bar
R_omega2_omega2j_bar = R_omega2*R_omega2j_bar

# Get back the eulers
omega1i_bar_p = my_euler_from_matrix(R_omega1_omega1i_bar); #print 'omega1i_bar_p', omega1i_bar_p
omega2j_bar_p = my_euler_from_matrix(R_omega2_omega2j_bar); #print omega2j_bar_p

# Expand using these new euler angles
rij = expand_in_R(l1max,l2max,ri,rj,omega1i_bar_p,omega2j_bar_p,R)
rij = rij.expand(complex=True)
rij = eval(str(rij))
print rij

In [ ]:
# Now we try an expansion of the Wigner D-matrices instead

# This relates the two frames
R = R_num

# Specify rotations of each frame
omega1 = [alpha1_num,beta1_num,gamma1_num];
omega2 = [alpha2_num,beta2_num,gamma2_num]; 

# Expand
rij = expand_in_omega_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R)
rij = rij.expand(complex=True)
rij = eval(str(rij))
rij = rij.subs(pi, 3.1415926535)
print rij


In [ ]:
# Now we try an expansion of the Wigner D-matrices, with angle psi applied to both

# This relates the two frames
R = R_num
psi = psi_num

# Specify rotations of each frame
omega1 = [alpha1_num,beta1_num,gamma1_num];
omega2 = [alpha2_num,beta2_num,gamma2_num]; 

# Expand ... if psi=0, this result should equal the box above
rij, brackets = expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R,psi)
rij = eval(str(rij))
print rij

In [ ]:
# Now we try the same with symbolic variables

# This relates the two frames
R = R_sym
psi = psi_sym

# Specify rotations of each frame
omega1 = [alpha1_sym, beta1_sym, gamma1_sym];
omega2 = [alpha2_sym, beta2_sym, gamma2_sym]; 

# Expand
rij_sym, brackets_sym = expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R,psi)
rij_sym = rij_sym.expand(complex=True)
brackets_sym = brackets_sym.expand(complex=True)
#print rij_sym

In [ ]:
# Evaluate for numerical values of R, psi, etc ... this should equal the "expand_in_omega_R_psi" result above
rij1 = rij_sym.subs([ \
                  (R_sym,R_num),\
                  (psi_sym,psi_num),\
                  (alpha1_sym,alpha1_num),\
                  (beta1_sym,beta1_num),\
                  (gamma1_sym,gamma1_num),\
                  (alpha2_sym,alpha2_num),\
                  (beta2_sym,beta2_num),\
                  (gamma2_sym,gamma1_num),\
               ]
               )
rij1 = eval(str(rij1))
print rij1

In [ ]:
# This is to look at the expansion of the term in brackets
alpha1, beta1, gamma1 = symbols('alpha1 beta1 gamma1', real=True)
alpha2, beta2, gamma2 = symbols('alpha2 beta2 gamma2', real=True)
R, psi = symbols('R psi', real=True)

# These substitutions are just for aesthetics
brackets_sym = brackets_sym.subs([ \
                  (R_sym,R),\
                  (psi_sym,psi),\
                  (alpha1_sym,alpha1),\
                  (beta1_sym,beta1),\
                  (gamma1_sym,gamma1),\
                  (alpha2_sym,alpha2),\
                  (beta2_sym,beta2),\
                  (gamma2_sym,gamma1),\
                 ])

# Print the term in brackets
sympy.pretty_print(brackets_sym)

# Print terms according to powers of 1/R
x = symbols('x', real=True)
test = brackets_sym.subs(R, 1/x)
for i in range(3):
    term = test.diff(x,i).subs(x,0)/math.factorial(i)
    print "Coefficient of 1/R^", i
    #sympy.pretty_print(term)
    print(term)
    print "\n"
